# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [78]:
# all import statements needed for the project, for example:
import sys
#!{sys.executable} -m pip install mss
#!{sys.executable} -m pip install Keras
#!{sys.executable} -m pip install pynput
#!{sys.executable} -m pip install setuptools
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install Pillow
#!pip install --upgrade --force-reinstall pillow


import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from os.path import exists

In [79]:
# any general notebook setup, like log formatting

In [80]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [81]:
def calculate_distance(from_coord, to_coord):
    x1 = float(from_coord[0])
    y1 = float(from_coord[1])
    x2 = float(to_coord[0])
    y2 = float(to_coord[1])
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    


In [82]:
def test_calculate_distance():
    p1 = (3,5)
    p2 = (6,2)
    dist = calculate_distance(p1,p2)
    print(dist)

In [92]:
def test(x):
    return calculate_distance((x["pickup_latitude"],x["pickup_longitude"]),(x["dropoff_latitude"],x["dropoff_longitude"]))

def add_distance_column(dataframe):
    new_column_values = []
    for index, x in dataframe.iterrows():
        new_column_values.append(calculate_distance((x["pickup_latitude"],x["pickup_longitude"]),(x["dropoff_latitude"],x["dropoff_longitude"])))
    dataframe.insert(loc=len(dataframe.columns) ,column='distance',value=new_column_values)
    
    #dataframe = dataframe.apply(test) #distance=lambda x: x["pickup_latitude"]+x["pickup_longitude"])#calculate_distance((x["pickup_latitude"],x["pickup_longitude"]),(x["dropoff_latitude"],x["dropoff_longitude"])))
    return dataframe

### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [97]:
def find_taxi_csv_urls():
    years = [2009,2010,2011,2012,2013,2014,2015]
    content = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(content.text, 'html.parser')
    divs = soup.find_all("div")
    new_divs1 = []
    new_divs2 = []
    
    ancors = soup.find_all("a")
    ancors_yellow = []
    
    for i in ancors:
        if 'title' in i.attrs.keys() and i["title"] == "Yellow Taxi Trip Records" and int(i["href"][-11:-7]) in years:
            ancors_yellow.append(i["href"])
    
    return ancors_yellow
    

def get_and_clean_month_taxi_data(url):
    name = url[url.rfind("/")+1:]
    
    if(exists("./taxi_csv/"+name)):
        clean_taxi_file('./taxi_csv/'+name)
    else:
        r = requests.get(url)
        with open('./taxi_csv/'+name, 'wb') as f:
            f.write(r.content)
            
        clean_taxi_file('./taxi_csv/'+name)
            
    
def clean_taxi_file(filename):
    data = pd.read_csv(filename)
    df = pd.DataFrame(data)
    df = df.drop(columns=["VendorID","passenger_count","trip_distance","RateCodeID","store_and_fwd_flag","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount"], axis=1)
    p1 = NEW_YORK_BOX_COORDS[0]
    p2 = NEW_YORK_BOX_COORDS[1]
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    
    df = df[(df["pickup_latitude"] > x1) & (df["pickup_latitude"] < x2) & (df["pickup_longitude"] > y1) & (df["pickup_longitude"] < y2)]
    df = df[(df["dropoff_latitude"] > x1) & (df["dropoff_latitude"] < x2) & (df["dropoff_longitude"] > y1) & (df["dropoff_longitude"] < y2)]
    print(df.head)
    print(df.columns)
    df = df.dropna()
    print(df.head)
    print(df.columns)
    df = add_distance_column(df)
    df = df.drop(columns=["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"], axis=1)
    
    df.to_csv(filename)  
    
    print(df.head)
    print(df.columns)



    
    #data_frame.loc[:,["Lectures","Grades"]]
    
    return pd.DataFrame()

    f = open(url,"r")
    dataframe = pd.read_csv(url)
    

def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()[0:3] # done
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data
#find_taxi_csv_urls()
x = get_and_clean_month_taxi_data("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv")
print(x)

C:\Users\rabee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


KeyError: "['RateCodeID'] not found in axis"

### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [26]:
def load_and_clean_uber_data(csv_file):
    raise NotImplemented()
    
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [10]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplemented()
    
    
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [17]:
def dataframes_to_table():
    raise NotImplemented()

In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

dataframes_to_table(table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)